In [1]:
import pandas as pd
import re
import time
import re
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
#read data
binary_data=pd.read_csv('text4cls.csv')
vote_text=binary_data['text']
help_text=binary_data['help']

In [3]:
#remove \t\n
def preprocess(x):
    a=re.sub(r'\t*|\n','',str(x)).split('\r')
    b=[]
    for i in a:
        b.append(re.sub(r'Posted:','*',i).split('*')[0].strip())
    b=[b for b in b if b]
    return b[0]

In [4]:
preprocess(vote_text[0])

'Refined gameplay, with layers of mechanics, on top of a traditional clicker. Kept me entertained for hours.Although a new product on Steam, I can tell this game has been through many iterations of improvements. And I really connected with the weird mysticism theme, too. The secondary core mechanic is capturing void entities and then spamming class-specific spells at them. I really enjoyed that part of the game, and it does require some thinking about spell combinations. Since Idle Wizard is a free to play game, it was easy to get started, and the "pay for tokens" system did not get in my way at all.This game uses the unity game engine, and it worked fine on my ubuntu linux desktop. I did use the ubuntu tool "xdotool" to speed along my mouseclicking. Even with this "cheat", this game was way, way, waaaaay too slow for me. After several hours, I had only advanced to unlock a specific class and obtain the first couple spells. I am sure that is okay for many people who leave it running in

In [ ]:
#clean help 
clean_vote_help=[]
for i in range(len(help_text)):
    clean_vote_help.append(preprocess(help_text[i]))
#clean vote and make empty cell for null value   
clean_vote_text=[]
for i in range(len(vote_text)):
    try:
        clean_vote_text.append(preprocess(vote_text[i]))
    except IndexError:
        clean_vote_text.append('')    

In [ ]:
#make a copy of 100000 data
new_vote=binary_data.copy()
new_vote['text']=clean_vote_text
new_vote['help']=clean_vote_help
new_vote=new_vote.drop(columns="Unnamed: 0")

In [ ]:
#replace empty cell to NaN
new_vote=new_vote.replace(r'^\s*$', np.nan, regex=True)
#drop all NaN
new_vote=new_vote.dropna()

In [ ]:
#text normalization
def text_normalizer(text):
    text=text.lower()
    my_stopwords=set(stopwords.words('english')+list(string.punctuation)+["can't",'would','u','cant','dont','im','ive'])
    text=re.sub('[^A-Za-z]+', ' ', text)
    text=text.split(' ')
    text=[w for w in text if w not in my_stopwords]
    text=[w for w in text if w]
    porter=nltk.PorterStemmer()   
    text=[porter.stem(t) for t in text]
    return ' '.join(text)

In [ ]:
#reset index 
new_vote = new_vote.reset_index(drop=True)

In [ ]:
#normalize each review 
final_vote_word=[]
for i in range(len(new_vote['text'])):
    final_vote_word.append(text_normalizer(new_vote['text'][i]))

In [ ]:
#convert vote to recommend with 0 1
final_vote['recommend']=[1 if i=='Recommended' else 0 for i in final_vote['vote']]

In [ ]:
#extract helpful and funny
helpful=[]
funny=[]
for i in final_vote['help']:
    i=re.sub(r'[^\d]+',' ',i).split(' ')
    helpful.append(str(i[0]))
    funny.append(str(i[1]))

In [ ]:
#make a copy of new_vote and combine necessary columns
final_vote=new_vote.copy()
final_vote['text']=final_vote_word
final_vote['help']=new_vote['help']
final_vote['recommend']=[1 if i=='Recommended' else 0 for i in final_vote['vote']]
final_vote['helpful']=['0' if i=='' else i for i in helpful]
final_vote['funny']=['0' if i=='' else i for i in funny]
final_vote.to_csv('clean_vote_vect.csv', index=False)

In [5]:
#read the data
final_vote=pd.read_csv('clean_vote_vect.csv')

In [9]:
final_vote.shape

(101871, 6)

In [10]:
#split training and testing
X_train,X_test,y_train,y_test=train_test_split(final_vote['text'],
                                              final_vote['recommend'],
                                              test_size=0.25,
                                              random_state=0)
X_train,X_val,y_train,y_val=train_test_split(X_train,
                                             y_train,
                                              test_size=0.25,
                                              random_state=0)

In [11]:
X_test

54473     potenti great game quit yet tho alway wish mod...
90774                            scp simul could use peanut
59802              jump helicarri hulk land forc game crash
80135     banish game focus exclus citi build resourc ma...
95394     board game variou softwar implement probabl pl...
6924      point game mix feel recommend bare posit reall...
4363        want play simul need great pc beest choic trust
100875    like game trion world big let frist block ip r...
47728     love subnautica insan graphic support earli ac...
45747                                         fun game good
83728     starter play game steam releas replay steam eb...
42187     hello everybodi play game want wast timehahahahah
61947       love super intuit nice get support vive control
99831     come stronghold crusad game realli love fade a...
11216     game oh game oooooooh game ooooooooooooooooooo...
9745      softwar great seem work well low end pc amd cp...
29860     major review state game recycl

In [12]:
#set type
X_train=X_train.values.astype('U')
X_test=X_test.values.astype('U')

In [16]:
### 1-gram
vect=CountVectorizer(ngram_range=(1,1),min_df=5).fit(X_train)
vect_name=vect.get_feature_names()
X_train_vectorized = vect.transform(X_train)
full_text_matrix=vect.transform(final_vote['text'].values.astype('U'))

In [17]:
#sparce matrix
text_vector=pd.DataFrame(full_text_matrix.toarray(),columns=vect.get_feature_names())
text_vector.head()

,aa,aaa,aaaand,aaand,ab,aback,abandon,abandonwar,abbrevi,abduct,...,zk,zombi,zomboid,zone,zoo,zoom,ztd,zu,zula,zweihand
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#logistic regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
score_pred = model.predict(vect.transform(X_test))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [20]:
#results for logstic regression
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score
print(confusion_matrix(y_test,score_pred))
print(f1_score(y_test,score_pred))
print(accuracy_score(y_test,score_pred))

[[ 3912  2202]
 [  936 18418]]
0.9214989743333167
0.8767865556777132
